# Filtro de Kalman para estimar la inclinación del chasis

El objetivo es estimar lo mejor posible el ángulo de inclinación del chasis $\theta$, utilizando únicamente la información de tres sensores: los ejes $x$ y $z$ del acelerómetro, y el eje $y$ del giróscopo. En este caso, podríamos conseguir una estimación más precisa utilizando el modelo completo del sistema, ya que disponemos también de *encoders* en los motores, pero eso requeriría programar un observador de estado innecesariamente complejo para este proyecto. Si se considera el sensor inercial como un sistema aislado, se puede obtener un filtro de Kalman muy sencillo, que proporciona una estimación del ángulo perfectamente válida para controlar el robot. Al no tener en cuenta el movimiento del sistema completo, tendremos que suponer que el acelerómetro sólo mide la aceleración de la gravedad, tratando cualquier otra aceleración como ruido del sensor.

![Esquema del sistema](./Images/figure-05.png)

## Modelo del sistema

El filtro de Kalman que utilizaremos tiene un solo estado, que es el ángulo de inclinación $\theta$, con un *input*, que es la velocidad angular del chasis $\omega_y$, y dos sensores, que son las aceleraciones según los ejes $x$ y $z$ locales. La ecuación de propagación de estados en forma discreta, para un paso de tiempo constante $\Delta t$, tendrá entonces la siguiente forma, con $A = \left[1\right]$ y $B = \left[\Delta t\right]$:

$$
    \theta_{n+1} = A\theta_n + B\left(\omega_{yn} + v_n\right)
$$

El término $v$ es el ruido del giróscopo, que consideraremos ruido blanco Gaussiano de varianza $\sigma_v^2$. Al introducir el giróscopo como *input*, nos evitamos tener que duplicar estados: si el giróscopo se tratase como un sensor, el vector de estados tendría que contener también la velocidad angular $\dot\theta$, porque sería necesaria para la función de observación $\mathbf h$.

Si partimos del supuesto de que el acelerómetro sólo percibe la gravedad, podemos establecer la siguiente expresión para la función de observación, considerando aceleraciones medidas en $g$:

$$
    \mathbf{h}_n = \begin{bmatrix}\cos\theta_n \\ \sin\theta_n\end{bmatrix} + \mathbf w_n
$$

donde $\mathbf w$ es un vector que contiene el ruido de los dos acelerómetros. Aunque no sea estrictamente cierto, porque se están incluyendo en $\mathbf w$ las aceleraciones debidas al movimiento del sensor, asumiremos que ambas señales contienen ruido blanco Gaussiano de varianza $\sigma_a^2$. La función $\mathbf h$ es no lineal, así que tendremos que utilizar un EKF, donde la matriz Jacobiana de la función de observación será:

$$
    \mathbf{H}_n = \begin{bmatrix}-\sin\theta_n \\ \cos\theta_n\end{bmatrix}
$$

La covarianza del ruido del sistema, teniendo en cuenta que el ruido del giróscopo va afectado por una ganancia $\Delta t$ en la ecuación de estados, es:

$$
    Q = \Delta_t^2\sigma_v^2
$$

Y la matriz de covarianza del ruido de los acelerómetros, si suponemos que son independientes entre sí, es:

$$
    \mathbf R = \mathbf I_2\sigma_a^2
$$

donde $\mathbf I_2$ es una matriz identidad de 2 x 2.

In [ ]:
from sympy import symbols, Symbol, Matrix, sin, cos, solve, simplify, eye, sqrt

theta, ax, az, gy, P = symbols('theta a_x a_z omega_y P')
sv, sa = symbols('sigma_v sigma_a', real=True, positive=True)
dt = Symbol('\Delta t', real=True, positive=True)

A = Matrix([1])                          # Matriz de propagación de estados
B = Matrix([dt])                         # Matriz de entrada

z = Matrix([[ax], [az]])                 # Vector de sensores (ejes x y z del acelerómetro)
h = Matrix([[cos(theta)], [sin(theta)]]) # Función de sensores
H = h.diff(theta)                        # Matriz Jacobiana de la función de sensores

Q = Matrix([(dt*sv)**2])                 # Matriz de covarianza del ruido del sistema
R = eye(2)*sa**2                         # Matriz de covarianza del ruido de los sensores

## Filtro de Kalman Extendido (EKF)

Las ecuaciones del EKF serán, teniendo en cuenta que en nuestro caso $A$, $B$, $Q$ y $\mathbf R$ son constantes:

 * Estimación *a priori* del estado:
 
 $$
     \hat\theta_{n+1}^- = A\hat\theta_n + B\omega_{yn}
 $$

 * Valor *a priori* de la covarianza de la estimación:

 $$
    P_{n+1}^- = AP_nA^{\mathsf T} + Q
 $$

 * Ganancia del filtro:

 $$
    \mathbf K_{n+1} = P_{n+1}^-\mathbf H_{n+1}^{-\mathsf T}\left(\mathbf H_{n+1}^-P_{n+1}^-\mathbf H_{n+1}^{-\mathsf T} + \mathbf R\right)^{-1}
 $$

 * Corrección para obtener la estimación *a posteriori* del estado:
 
 $$
     \hat\theta_{n+1} = \hat\theta_{n+1}^- + \mathbf K_{n+1}\left(\mathbf z_{n+1} - \mathbf h_{n+1}^-\right)
 $$
  
 * Valor *a posteriori* de la covarianza de la estimación:
 
 $$
     P_{n+1} = \left(1 - \mathbf K_{n+1}\mathbf H_{n+1}^-\right)P_{n+1}^-
 $$
 

## Filtro simplificado aprovechando la convergencia de $P$

En este filtro, se puede comprobar que la covarianza de la estimación del estado $P$ converge a un valor constante al cabo de unas cuantas iteraciones. Este hecho nos va a resultar útil para simplificar el algoritmo y hacerlo más eficiente, como se verá a continuación. Lo primero que tenemos que hacer es expresar $P_{n+1}$ como función de $P_n$:

In [ ]:
# Estimación a priori
P1 = A*P*A.T + Q

# Ganancia del filtro
K = P1*H.T*(H*P1*H.T + R).inv()

# Estimación a posteriori
P2 = simplify((eye(1) - K*H)*P1)

$$
    P_{n+1} = \frac{\sigma_a^2\left(P_n + \Delta t^2\sigma_v^2\right)}{P_n + \Delta t^2\sigma_v^2 + \sigma_a^2}
$$

Supondremos que, cuando $n$ sea suficientemente grande, $P$ convergerá a un valor constante $P_\infty$. Eso quiere decir que tanto $P_n$ como $P_{n+1}$ alcanzarán dicho valor, así que podemos sustituirlos en la ecuación anterior, y resolverla después para obtener $P_\infty$:

In [ ]:
# Convergencia: P(n) = P(n + 1) si 'n' tiende a infinito
Pinf = solve(P - P2[0], P)[0]

La ecuación tiene solución real, de modo que, efectivamente, se cumple que $P$ converge a un valor constante $P_\infty$:

$$
    P_\infty = \frac{1}{2}\left(\sqrt{\Delta t^4\sigma_v^4 + 4\Delta t^2\sigma_v^2\sigma_a^2} - \Delta t^2\sigma_v^2\right)
$$

Ahora estudiamos la estimación *a posteriori* del estado, mirando qué forma toma la corrección:

In [ ]:
# Corrección del estado
dx = simplify(K*(z - h))

$$
    \mathbf K_{n+1}\left(\mathbf z_{n+1} - \mathbf h_{n+1}^-\right) = \frac{P_n + \Delta t^2\sigma_v^2}{P_n + \Delta t^2\sigma_v^2 + \sigma_a^2}\left(\bar a_{zn+1}\cos\hat\theta_{n+1}^- - \bar a_{xn+1}\sin\hat\theta_{n+1}^-\right)
$$

Cuando el filtro converge, $P_n$ alcanza el valor constante $P_\infty$, de modo que podemos expresar la corrección como:

$$
    \mathbf K_{n+1}\left(\mathbf z_{n+1} - \mathbf h_{n+1}^-\right) = C_\infty\left(\bar a_{zn+1}\cos\hat\theta_{n+1}^- - \bar a_{xn+1}\sin\hat\theta_{n+1}^-\right)
$$

donde la constante $C_\infty$ es una función de las varianzas y el paso de tiempo:

$$
    C_\infty = \frac{P_\infty + \Delta t^2\sigma_v^2}{P_\infty + \Delta t^2\sigma_v^2 + \sigma_a^2} = \frac{P_\infty}{\sigma_a^2}
$$

Esta expresión se puede linealizar para pasos de tiempo pequeños, resultando:

In [ ]:
Cinf = Pinf/sa**2
C0 = Cinf.subs(dt, 0) + Cinf.diff(dt).subs(dt, 0)*dt

$$
    C_\infty \approx \frac{\sigma_v}{\sigma_a}\Delta t
$$

Se puede comprobar que, cuando $\sigma_v$ es menor que $\sigma_a$, esta aproximación mantiene su validez para pasos de tiempo bastante elevados. Esto es precisamente lo que ocurre en nuestro sistema: $\sigma_v^2$ es la varianza del ruido del giróscopo, que debería ser pequeña, mientras que $\sigma_a^2$ incluye las aceleraciones no debidas a la gravedad, que serán de una magnitud mucho mayor que el ruido propio de los sensores. 

## Implementación en el microcontrolador

En la práctica, no necesitaremos conocer las varianzas del ruido del giróscopo y los acelerómetros. Bastará con establecer que el parámetro $C_\infty$ es igual a una constante $K$ multiplicada por el paso de tiempo:

$$
    C_\infty \approx K\Delta t
$$

La constante $K$ es un parámetro que ajustaremos manualmente para conseguir el comportamiento deseado: un valor bajo de $K$ dará mayor peso al giróscopo, mientras que un valor elevado aplicará con mas intensidad la corrección de los acelerómetros.

En resumen, el filtro de Kalman que se utilizará para estimar el ángulo de inclinación del robot consistirá simplemente en dos operaciones:

$$
\begin{align}
    \hat\theta_{n+1}^- &= \hat\theta_n + \omega_{yn}\Delta t \\
    \hat\theta_{n+1} &= \hat\theta_{n+1}^- + K\Delta t\left(\bar a_{zn+1}\cos\hat\theta_{n+1}^- - \bar a_{xn+1}\sin\hat\theta_{n+1}^-\right)
\end{align}    
$$

En cada vuelta del bucle de control, primero se integra un paso de tiempo utilizando la lectura del giróscopo, y luego se aplica la corrección usando las aceleraciones.